In [ ]:
#pip install selenium pandas openpyxl

In [ ]:
import time
import pandas as pd
from datetime import datetime
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# เปิด browser ใหม่
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--remote-debugging-port=9222") 
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

# ห่อหุ้มการทำงานหลักทั้งหมดด้วย try...finally เพื่อให้แน่ใจว่าเบราว์เซอร์จะปิดเสมอ
try:
    # โหลด username และ password
    username = Path("username.txt").read_text(encoding="utf-8").strip()
    password = Path("password.txt").read_text(encoding="utf-8").strip()

    # เข้าหน้า login
    driver.get("https://gpos_web-1pq8t2.th.biot-apps.com/login")
    
    # กรอก username / password
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='text']"))).send_keys(username)
    driver.find_element(By.CSS_SELECTOR, "input[type='password']").send_keys(password)

    # ติ๊ก checkbox ที่อยู่ล่างสุด (นโยบายความเป็นส่วนตัว)
    try:
        time.sleep(1)
        checkboxes = driver.find_elements(By.CSS_SELECTOR, "input[type='checkbox']")
        if checkboxes:
            driver.execute_script("arguments[0].click();", checkboxes[-1])
            time.sleep(0.5)
        else:
            print("ไม่พบ checkbox นโยบายความเป็นส่วนตัว")
    except Exception as e:
        print(f"ไม่สามารถคลิก checkbox ได้: {e}")

    # กดปุ่มเข้าสู่ระบบ
    wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='เข้าสู่ระบบ']"))).click()
    print("กดปุ่ม 'เข้าสู่ระบบ'")

    # --- หลังจาก login แล้ว ---

    # 1. เลือกร้านค้า (กด "เข้า")
    try:
        enter_shop = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='เข้า']")))
        enter_shop.click()
        print("เลือกร้านค้าเรียบร้อย")
    except TimeoutException:
        print("ไม่พบปุ่ม 'เข้า' ร้านค้า อาจเข้าอัตโนมัติแล้ว")

    # ปิด popup “เข้าใจแล้ว” ถ้ามี
    try:
        popup_button = WebDriverWait(driver, 8).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='เข้าใจแล้ว']"))
        )
        popup_button.click()
        print("คลิกปุ่ม 'เข้าใจแล้ว' เรียบร้อย")
    except TimeoutException:
        print("ไม่มี popup 'เข้าใจแล้ว' แสดง")
        
    print("รอให้หน้าหลัก (Dashboard) โหลด...")
    wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'รายงาน')]")))
    print("หน้าหลักโหลดสำเร็จ")

    # คลิก "จัดการออร์เดอร์"
    try:
        order_menu_span = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'จัดการออร์เดอร์')]")))
        order_menu_span.click()
        print("คลิกเมนู 'จัดการออร์เดอร์'")
    except Exception as e:
        print(f"เมนู 'จัดการออร์เดอร์' ไม่เจอ หรือคลิกไม่ได้: {e}")
        raise

    # คลิก "ออเดอร์ทั้งหมด"
    try:
        order_all_span = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='ออเดอร์ทั้งหมด']"))
        )
        order_all_span.click()
        print("คลิกเมนู 'ออเดอร์ทั้งหมด'")
    except Exception as e:
        print(f"เมนู 'ออเดอร์ทั้งหมด' ไม่เจอ หรือคลิกไม่ได้: {e}")
        raise
    
    # 1. รอให้หน้าโหลดและคลิกเลือก "เมื่อวาน"
    print("\nรอหน้า 'ออเดอร์ทั้งหมด' โหลดสมบูรณ์...")
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".ant-spin-spinning")))
    print("หน้า 'ออเดอร์ทั้งหมด' แสดงผลและพร้อมใช้งานแล้ว")
    time.sleep(2)
    try:
        # === จุดที่แก้ไข: ทำให้การคลิกปฏิทินและ 'เมื่อวาน' เสถียรขึ้น ===
        print("กำลังคลิกปฏิทิน...")
        date_picker_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "(//div[contains(@class, 'maxDateRangePicker-root')]//button)[1]")
        ))
        date_picker_button.click()
        print("คลิกปฏิทินสำเร็จ")
        
        time.sleep(0.5) # หน่วงเวลาเล็กน้อยเพื่อให้แน่ใจว่าเมนูวันที่แสดงขึ้นมาสมบูรณ์
        
        print("กำลังเลือก 'เมื่อวาน'...")
        yesterday_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[text()='เมื่อวาน']")))
        yesterday_button.click()
        print("เลือกวันที่ 'เมื่อวาน' สำเร็จ")

    except Exception as date_error:
        print(f"ไม่สามารถเลือกวันที่ได้: {date_error}")
        raise 

    print("กำลังโหลดข้อมูลของเมื่อวาน...")
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".ant-spin-spinning")))
    time.sleep(1) # รอให้ DOM นิ่งหลังจากโหลดเสร็จ
    print("โหลดข้อมูลของเมื่อวานเรียบร้อย")

    # 2. เริ่มต้นกระบวนการเก็บข้อมูล
    all_orders_data = []
    page_number = 1
    last_first_order_id = None
    
    ORDER_ITEM_SELECTOR = (By.XPATH, "//div[contains(@style, 'padding: 12px 0px 0px')]")

    while True:
        print(f"\n--- กำลังประมวลผลหน้า {page_number} ---")
        
        try:
            wait.until(EC.presence_of_element_located(ORDER_ITEM_SELECTOR))
            first_order_on_page = driver.find_element(*ORDER_ITEM_SELECTOR)
            current_first_order_id = first_order_on_page.text
            
            if current_first_order_id == last_first_order_id:
                print("ตรวจพบข้อมูลหน้าซ้ำซ้อน น่าจะถึงหน้าสุดท้ายแล้ว สิ้นสุดการทำงาน")
                break
            
            last_first_order_id = current_first_order_id
            order_count = len(driver.find_elements(*ORDER_ITEM_SELECTOR))
            print(f"พบ {order_count} ออเดอร์ในหน้าปัจจุบัน")

        except TimeoutException:
            print("ไม่พบรายการออเดอร์ในหน้านี้ (อาจเป็นหน้าสุดท้าย)")
            break

        for i in range(order_count):
            try:
                orders_to_process = driver.find_elements(*ORDER_ITEM_SELECTOR)
                if i >= len(orders_to_process):
                    print(f"  - Index {i} อยู่นอกช่วงของรายการออเดอร์, ข้ามไป...")
                    continue
                order_to_click = orders_to_process[i]
                
                payment_type_text = order_to_click.find_element(By.XPATH, ".//span[contains(text(), '฿')]/preceding-sibling::div/span").text.strip()
                payment_type = payment_type_text if payment_type_text else "N/A"

                driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", order_to_click)
                time.sleep(0.5)
                order_to_click.click()
                
                wait.until(EC.presence_of_element_located((By.XPATH, "//span[text()='รายละเอียดรายการ']")))
                time.sleep(0.5)

                receipt_no = wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'เลขที่ใบเสร็จ')]/following-sibling::span"))).text.strip()
                date_val = driver.find_element(By.XPATH, "//span[contains(text(), 'วันที่')]/following-sibling::span").text.strip()
                time_val_full = driver.find_element(By.XPATH, "//span[contains(text(), 'เวลา')]/following-sibling::span").text.strip()
                time_val = time_val_full.split(' ')[1] if ' ' in time_val_full else time_val_full
                
                table_no = driver.find_element(By.XPATH, "//span[contains(text(), 'หมายเลขโต๊ะ')]/following-sibling::span").text.strip()
                customer_count_text = driver.find_element(By.XPATH, "//span[contains(text(), 'ลูกค้า')]").text
                customer_count = ''.join(filter(str.isdigit, customer_count_text)) or 'N/A'
                
                total_amount = driver.find_element(By.XPATH, "//div[contains(@style, 'position: sticky')]//span[contains(text(), '฿')]").text.replace('฿', '').strip()
                
                item_rows_xpath = "//span[text()='รายละเอียดรายการ']/ancestor::div[1]/following-sibling::div/div"
                item_rows = driver.find_elements(By.XPATH, item_rows_xpath)
                
                item_count_in_order = 0
                for item in item_rows:
                    try:
                        name_container = item.find_element(By.XPATH, "./div/div[1]")
                        item_name = name_container.text.strip().replace('\n', ' ')
                        
                        quantity_container = item.find_element(By.XPATH, "./div/div[2]")
                        item_quantity = quantity_container.text.strip().replace('x', '').strip()
                        
                        price_container = item.find_element(By.XPATH, "./div/div[3]")
                        item_price = price_container.text.replace('฿', '').strip()

                        all_orders_data.append({
                            "เลขที่ใบเสร็จ": receipt_no, "วันที่": date_val, "เวลา": time_val,
                            "หมายเลขโต๊ะ": table_no, "จำนวนลูกค้า": customer_count,
                            "รายละเอียดรายการ": item_name, "จำนวน": item_quantity,
                            "ราคา": item_price, "ยอดรวมทั้งหมด": total_amount,
                            "ประเภทการชำระเงิน": payment_type,
                        })
                        item_count_in_order += 1
                    except NoSuchElementException:
                        continue
                
                print(f"  - เก็บข้อมูลออเดอร์ {receipt_no if receipt_no else '--'} สำเร็จ ({item_count_in_order} รายการ)")

            except Exception as e:
                print(f"เกิดข้อผิดพลาดในการดึงข้อมูลออเดอร์ที่ {i + 1}: {e}")
                error_screenshot_filename = f"error_order_{page_number}_{i+1}.png"
                driver.save_screenshot(error_screenshot_filename)
                print(f"บันทึกภาพหน้าจอที่ '{error_screenshot_filename}'")
                continue

        try:
            print("กำลังเลื่อนไปที่ส่วนท้ายของหน้า...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

            next_button_xpath = "//span[text()='ต่อไป']/ancestor::div[contains(@style, 'cursor: pointer')]"
            next_button = wait.until(EC.presence_of_element_located((By.XPATH, next_button_xpath)))
            
            next_button_span = next_button.find_element(By.XPATH, ".//span")
            if "rgb(153, 153, 153)" in next_button_span.value_of_css_property('color'):
                 print("\nปุ่ม 'ถัดไป' ถูกปิดใช้งานแล้ว ถึงหน้าสุดท้าย")
                 break
            
            print("\nกำลังไปหน้าถัดไป...")
            driver.execute_script("arguments[0].click();", next_button)
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, ".ant-spin-spinning")))
            time.sleep(1.5) # รอให้ DOM ของหน้าใหม่นิ่งสนิท
            print("โหลดหน้าถัดไปเรียบร้อย")
            page_number += 1
        except (NoSuchElementException, TimeoutException):
            print("\nไม่พบปุ่ม 'ถัดไป' หรือปุ่มไม่พร้อมใช้งาน, สิ้นสุดการทำงาน")
            break

    # 4. บันทึกข้อมูลเป็นไฟล์ Excel
    if all_orders_data:
        df = pd.DataFrame(all_orders_data)
        file_date = datetime.now().strftime("%Y-%m-%d")
        filename = f"gpos_{file_date}.xlsx"
        df.to_excel(filename, index=False, engine='openpyxl')
        print(f"\nบันทึกข้อมูลทั้งหมด {len(df)} แถว ลงในไฟล์ '{filename}' เรียบร้อยแล้ว")
    else:
        print("\nไม่พบข้อมูลของเมื่อวานให้บันทึก")

except Exception as e:
    print(f"\nกิดข้อผิดพลาดร้ายแรงในกระบวนการทำงาน: {e}")
    try:
        error_filename = "error_page.html"
        with open(error_filename, "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print(f"ได้บันทึกหน้าเว็บที่เกิดปัญหาลงในไฟล์ '{error_filename}' แล้ว")
    except Exception as save_error:
        print(f"   - ไม่สามารถบันทึกหน้าเว็บที่เกิดปัญหาได้: {save_error}")
finally:
    if 'driver' in locals() and driver:
        driver.quit()
        print("ปิดเบราว์เซอร์เรียบร้อย")
